In [0]:
import sys
import logging

import time
import numpy as np
import pandas as pd
import os
import csv

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from xgboost import XGBClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l1_l2
import pickle

## combine adaboost, catboost, NN, LGBM, xgboost 

In [106]:
## Catboost
model1 = pd.read_csv("applied/otto/level2_model/level3/level2_Catboost_final.csv", sep=',', header=0)
## NN
model2 = pd.read_csv("applied/otto/level2_model/level3/level_2_NN_final.csv", sep=',', header=0)
## LGBM
model3 = pd.read_csv("applied/otto/level2_model/level3/level2_model_6_LGBM.csv", sep=',', header=0)
## xgboost
model4 = pd.read_csv("applied/otto/level2_model/level3/level2_Xgboost.csv", sep=',', header=None)

level3= np.zeros([144368, 9, 4])
level3[:,:,0] = model1.values[:,1:]
level3[:,:,1] = model2.values[:,1:]
level3[:,:,2] = model3.values[:,1:]
level3[:,:,3] = model4.values
weight = [10, 10, 45, 35]

level3_weighted =  (level3[:,:,0]*weight[0] + level3[:,:,1]*weight[1] + level3[:,:,2] * weight[2] + level3[:,:,3]*weight[3]) / np.sum(weight)
print('weight sum: ', np.sum(weight))

weight sum:  100


In [109]:
# 要存的csv的地址
file_path = "D:/applied/otto/level2_model/assign_weights/"

count = 0
print(len(os.listdir(file_path)))
if len(os.listdir(file_path)) == 0:
     nth_file = str(1)
else:
    for i in range(len(os.listdir(file_path))):
        if  ('level3_weighted_result_' + str(i+1) + '.csv') in os.listdir(file_path):
          count += 1
        nth_file = str(count+1)

file_name = "level3_weighted_result_" + nth_file
submit = pd.DataFrame()
submit["id"] = np.array(range(1, level3.shape[0] + 1 ))
for j in range(9):
  submit["Class_" + str(j+1)] = level3_weighted[:,j]
submit.to_csv(file_path + file_name + ".csv", index = False)

parameter_file_path = file_path + "parameter.csv"

model_name = []
catboost_parameters = []
nn_parameters = []
lgbm_parameters = []
xgboost_parameters = []
loss = []

if os.path.exists(parameter_file_path):
    previous_parameters = pd.read_csv(parameter_file_path, sep=',', header=0)
    for i in range(previous_parameters.values.shape[0]):
        model_name.append(previous_parameters.values[i][0])
        catboost_parameters.append(previous_parameters.values[i][1])
        nn_parameters.append(previous_parameters.values[i][2])
        lgbm_parameters.append(previous_parameters.values[i][3])
        xgboost_parameters.append(previous_parameters.values[i][4])
        loss.append(previous_parameters.values[i][5])

catboost_parameters.append(weight[0])
nn_parameters.append(weight[1])
lgbm_parameters.append(weight[2])
xgboost_parameters.append(weight[3])
model_name.append(file_name)
loss.append(0)

submit = pd.DataFrame()
submit["model_name"] = model_name
submit["catboost"] = catboost_parameters
submit["nn"] = nn_parameters
submit["lgbm"] = lgbm_parameters
submit["xgboost"] = xgboost_parameters
submit["loss"] = loss
submit.to_csv(parameter_file_path, index = False)

4
